# Ensemble

Before running, make sure you have downloaded th ezip file for each model, extracted the model from it, and saved it in a directory named "model". Models can be access on [google drive](https://drive.google.com/drive/folders/1ldH5LbvOHNMP_eBM74szzdETlycwVdjW?usp=sharing).

In [61]:
import utils 
from tensorflow import keras
import pathlib
from  tensorflow.keras.metrics import mean_squared_error
import numpy as np


# LSTM

**Prepare data**

In [62]:
df = utils.load_and_clean_master_dataset()

df_train, df_val, df_test = utils.split_train_test_val(df)

X_train, y_train = utils.split_Xy(df_train)
X_val, y_val = utils.split_Xy(df_val)
X_test, y_test = utils.split_Xy(df_test)

X_train, scaler = utils.normalize(X_train, train=True)
X_val, _ = utils.normalize(X_val, train=False, scaler=scaler)
X_test, _ = utils.normalize(X_test, train=False, scaler=scaler)

i =7
X_train, X_val, X_test = X_train[utils.cols[:i+1]], X_val[utils.cols[:i+1]], X_test[utils.cols[:i+1]]


X_train, y_train = utils.to_batch(X_train, y_train)
X_val, y_val = utils.to_batch(X_val, y_val)
X_test, y_test = utils.to_batch(X_test, y_test)

**Reload model**

In [63]:
crnt_dir = pathlib.Path('.').parent.resolve()
lstm_model = keras.models.load_model(crnt_dir/'lstm'/'model')

**Evaluate model**

In [64]:
y_pred_lstm = lstm_model(X_test)
mean_squared_error(y_test.to_numpy(), y_pred_lstm.numpy().reshape(-1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.00086672825>

# Simple NN

**Prepare data**

In [84]:
df = utils.load_and_clean_master_dataset()

df_train, df_val, df_test = utils.split_train_test_val(df)

X_train, y_train = utils.split_Xy(df_train)
X_val, y_val = utils.split_Xy(df_val)
X_test, y_test = utils.split_Xy(df_test)

i = 5
X_train, X_val, X_test = X_train[utils.cols[:i+1]], X_val[utils.cols[:i+1]], X_test[utils.cols[:i+1]]


**Reload model**

In [85]:
crnt_dir = pathlib.Path('.').parent.resolve()
nn_model = keras.models.load_model(crnt_dir/'DNN'/'model')

**Evaluate model**

In [86]:
# Take into account sliding window of models
X_test, y_test = X_test[20:], y_test[20:]
y_pred_nn = np.array([p[0] for p in nn_model.predict(X_test)])
mean_squared_error(y_test.to_numpy(), y_pred_nn.reshape(-1))

2/2 [==============================] - 0s 5ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=0.00042526922>

# Transformer

**Prepare data**

In [12]:
df = utils.load_and_clean_master_dataset()

df_train, df_val, df_test = utils.split_train_test_val(df)

X_train, y_train = utils.split_Xy(df_train)
X_val, y_val = utils.split_Xy(df_val)
X_test, y_test = utils.split_Xy(df_test)

i = 4
X_train, X_val, X_test = X_train[utils.cols[:i+1]], X_val[utils.cols[:i+1]], X_test[utils.cols[:i+1]]

X_train, y_train = utils.to_batch(X_train, y_train)
X_val, y_val = utils.to_batch(X_val, y_val)
X_test, y_test = utils.to_batch(X_test, y_test)

**Reload model**

In [13]:
crnt_dir = pathlib.Path('.').parent.resolve()
transformer_model = keras.models.load_model(crnt_dir/'Transformer_model'/'model')

**Evaluate model**

In [14]:
y_pred_transformer = transformer_model(X_test)
mean_squared_error(y_test.to_numpy(), y_pred_transformer.numpy().reshape(-1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0004239239>

# Ensemble

In [95]:
y_pred = np.mean([y_pred_transformer, y_pred_nn.reshape((-1,1)), y_pred_lstm], axis=0)
ensemble_mse = mean_squared_error(y_test.to_numpy(), y_pred.reshape(-1))

In [97]:
print(f'Final (ensemble) MSE is {ensemble_mse}')

Final (ensemble) MSE is 0.0003023127210326493
